In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_hiking_trails(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Assuming each trail is listed in an HTML element with the class 'trail-info'
    trails = soup.find_all(class_='trail-info')
    
    trail_data = []
    for trail in trails:
        name = trail.find(class_='trail-name').text.strip() if trail.find(class_='trail-name') else 'No Name'
        location = trail.find(class_='trail-location').text.strip() if trail.find(class_='trail-location') else 'No Location'
        description = trail.find(class_='trail-description').text.strip() if trail.find(class_='trail-description') else 'No Description'
        
        trail_data.append({
            'Name': name,
            'Location': location,
            'Description': description
        })
    
    return trail_data

# Example URL - replace with the actual URL of your data source
url = "https://example.com/hiking-trails"
trails = scrape_hiking_trails(url)
print(trails)


In [3]:
import requests
from bs4 import BeautifulSoup

def scrape_trail_info(url):
    response = requests.get(url)
    # Create a BeautifulSoup object from the HTML response
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract trail name (assuming it's within an h1 tag for this example)
    trail_name = soup.find('h1').text.strip() if soup.find('h1') else 'No Title'
    
    # Initialize a dictionary to store the trail data
    trail_data = {'Name': trail_name}
    
    # Loop through each <dt> and <dd> pair to extract data
    for dt, dd in zip(soup.find_all('dt'), soup.find_all('dd')):
        label = dt.get_text().strip()
        value = dd.get_text().strip()
        
        # You can customize this part to format or clean the data as needed
        if label == "Beschreibung":
            trail_data['Description'] = value
        elif label == "Wegbeschreibung":
            trail_data['Route Description'] = value
        elif label == "Höchster Punkt":
            trail_data['Highest Point'] = value
        elif label == "Zielpunkt":
            trail_data['Destination'] = value

    return trail_data

# URL to scrape
url = "https://www.bergfex.de/sommer/bayern/touren/radfahren/24018,sagenhafter-weg--route-biessenhofen/"
trail_info = scrape_trail_info(url)
trail_info

{'Name': 'Sagenhafter Weg - Route Biessenhofen',
 'Description': 'Vier Routen, die durch Verbindungsstrecken kombiniert werden können, laden Sie ein zu einer Reise durch die Sagen- und Legendenwelt des Ostallgäus. Die spannenden Mythen - nähergebracht durch Infotafeln, Texte und lebensgroße Sagenfiguren - begleiten Sie durch die einladende Landschaft. Spielplätze, der Märchenweg am Elbsee, sowie Rastplätze versprechen einen angenehmen und anregenden Ausflug für die ganze Familie.',
 'Route Description': 'Die Runde startet in Biessenhofen und führt über die Südseite des Bachtelsees weiter nach Hörmannshofen und Ottilienberg. Von dort geht es von Kreen weiter nach Loretto und wieder zurück nach Kreen. Anschließend führt die Runde über Altdorf, Ebenhofen und Holdersberg zur Kapelle im Jungholz (Kapeller). Von dort geht es dann wieder zurück nach Biessenhofen.\nDie empfohlenen Fahrtrichtung ist im Uhrzeigersinn.\n\xa0\nDie Strecke besteht aus ca. 96 % Asphalt und etwa 4 % gekiesten Wirtsch

In [5]:
import pandas as pd

def extract_landmarks(description):
    # Simple extraction logic based on sentence ending
    # This is a placeholder, adjust the logic to fit the actual content pattern
    landmarks = []
    sentences = description.split('.')
    for sentence in sentences:
        if 'bei' in sentence or 'über' in sentence:
            landmarks.append(sentence.strip())
    return landmarks

# Create DataFrame from the dictionary
df = pd.DataFrame([trail_info])

df

,Name,Description,Route Description,Highest Point,Destination
0,Sagenhafter Weg - Route Biessenhofen,"Vier Routen, die durch Verbindungsstrecken kom...",Die Runde startet in Biessenhofen und führt üb...,749 m,Biessenhofen


In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

def scrape_trail_info(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    trail_name = soup.find('h1').text.strip() if soup.find('h1') else 'No Title'
    trail_data = {'Name': trail_name}
    
    for dt, dd in zip(soup.find_all('dt'), soup.find_all('dd')):
        label = dt.get_text().strip()
        value = dd.get_text().strip()
        if label == "Beschreibung":
            trail_data['Description'] = value
        elif label == "Wegbeschreibung":
            trail_data['Route Description'] = value
        elif label == "Höchster Punkt":
            trail_data['Highest Point'] = value
        elif label == "Zielpunkt":
            trail_data['Destination'] = value

    # Extract landmarks from the Route Description
    route_description = trail_data.get('Route Description', '')
    landmarks = extract_landmarks(route_description)
    trail_data['Landmarks'] = landmarks

    return trail_data

def extract_landmarks(description):
    # Use regex to find patterns of landmarks, assuming they are proper nouns or follow certain words
    pattern = r'\bbei\b\s(\w+)|\büber\b\s(\w+)|(\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*)'
    matches = re.findall(pattern, description)
    landmarks = [match[0] or match[1] or match[2] for match in matches if match]
    return landmarks

# URL to scrape
url = "https://www.bergfex.de/sommer/bayern/touren/radfahren/24018,sagenhafter-weg--route-biessenhofen/"
trail_info = scrape_trail_info(url)

# Create DataFrame from the dictionary
df = pd.DataFrame([trail_info])
df

,Name,Description,Route Description,Highest Point,Destination,Landmarks
0,Sagenhafter Weg - Route Biessenhofen,"Vier Routen, die durch Verbindungsstrecken kom...",Die Runde startet in Biessenhofen und führt üb...,749 m,Biessenhofen,"[Die Runde, Biessenhofen, die, Bachtelsees, Ot..."


In [7]:
df['Landmarks']

0    [Die Runde, Biessenhofen, die, Bachtelsees, Ot...
Name: Landmarks, dtype: object

In [8]:
import spacy

nlp = spacy.load('de_core_news_sm')

def extract_landmarks_nlp(description):
    doc = nlp(description)
    landmarks = [ent.text for ent in doc.ents if ent.label_ == 'LOC']
    return landmarks

In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

def scrape_trail_info(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    trail_name = soup.find('h1').text.strip() if soup.find('h1') else 'No Title'
    trail_data = {'Name': trail_name}
    
    for dt, dd in zip(soup.find_all('dt'), soup.find_all('dd')):
        label = dt.get_text().strip()
        value = dd.get_text().strip()
        if label == "Beschreibung":
            trail_data['Description'] = value
        elif label == "Wegbeschreibung":
            trail_data['Route Description'] = value
        elif label == "Höchster Punkt":
            trail_data['Highest Point'] = value
        elif label == "Zielpunkt":
            trail_data['Destination'] = value

    # Extract landmarks from the Route Description
    route_description = trail_data.get('Route Description', '')
    landmarks = extract_landmarks_nlp(route_description)
    trail_data['Landmarks'] = landmarks
    print(landmarks)

    return trail_data

# URL to scrape
url = "https://www.bergfex.de/sommer/bayern/touren/radfahren/24018,sagenhafter-weg--route-biessenhofen/"
trail_info = scrape_trail_info(url)

# Create DataFrame from the dictionary
df = pd.DataFrame([trail_info])
df

['Biessenhofen', 'Bachtelsees', 'Hörmannshofen', 'Ottilienberg', 'Loretto', 'Kreen', 'Altdorf', 'Ebenhofen', 'Holdersberg', 'Biessenhofen', 'Maibaum', 'Kaufbeurer Str.', 'Bachtelsee', 'OttilienbergLorettoEbenhofen', 'Schulhof', 'BaschtlehausKapelle', 'JungholzRamersberg', 'Ebenhofen', 'Bidingen', 'Bernbach']


,Name,Description,Route Description,Highest Point,Destination,Landmarks
0,Sagenhafter Weg - Route Biessenhofen,"Vier Routen, die durch Verbindungsstrecken kom...",Die Runde startet in Biessenhofen und führt üb...,749 m,Biessenhofen,"[Biessenhofen, Bachtelsees, Hörmannshofen, Ott..."
